In [1]:
import os

def fetch_reviews(path):
    data = []
    #path = 'aclImdb/train/pos/'
    files = [f for f in os.listdir(path)]
    for file in files:
        with open(path+file, "r", encoding='utf8') as f:
            data.append(f.read())
    return data

In [2]:
import pandas as pd

df_train_pos = pd.DataFrame({'review': fetch_reviews('C:\\Users\\HP\\Desktop\\CH\\aclImdb\\aclImdb\\train\\pos\\'), 'label': 1})
df_train_neg = pd.DataFrame({'review': fetch_reviews('C:\\Users\\HP\\Desktop\\CH\\aclImdb\\aclImdb\\train\\neg\\'), 'label': 0})
df_train_unsup = pd.DataFrame({'review': fetch_reviews('C:\\Users\\HP\\Desktop\\CH\\aclImdb\\aclImdb\\train\\unsup\\'), 'label': 2})


In [3]:
data = pd.concat([df_train_pos, df_train_neg, df_train_unsup], ignore_index=True)
print("Total reviews in df: ", data.shape)
data.head()

Total reviews in df:  (75000, 2)


,review,label
0,Bromwell High is a cartoon comedy. It ran at t...,1
1,Homelessness (or Houselessness as George Carli...,1
2,Brilliant over-acting by Lesley Ann Warren. Be...,1
3,This is easily the most underrated film inn th...,1
4,This is not the typical Mel Brooks film. It wa...,1


In [ ]:
data.tail()

In [4]:
data.groupby('label').label.count()

label
0    12500
1    12500
2    50000
Name: label, dtype: int64

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string
from sklearn.metrics import accuracy_score, confusion_matrix ,classification_report
from sklearn.model_selection import train_test_split 

In [6]:
def clean_text1(text):
    text=text.lower()
    text=re.sub('\[.*?\]','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub('\w*\d\w*','',text)
    return text

cleaned1=lambda x:clean_text1(x)
data['review']=pd.DataFrame(data.review.apply(cleaned1))
data.head()

,review,label
0,bromwell high is a cartoon comedy it ran at th...,1
1,homelessness or houselessness as george carlin...,1
2,brilliant overacting by lesley ann warren best...,1
3,this is easily the most underrated film inn th...,1
4,this is not the typical mel brooks film it was...,1


In [7]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [7]:
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
def stp_stm(text):
    stop_words=set(stopwords.words("english"))
    words=word_tokenize(text)
    words=[word for word in words if word not in stop_words]
    stemmer=SnowballStemmer("english")
    words=[stemmer.stem(word) for word in words]
    return " ".join(words)
data['review']=data['review'].apply(stp_stm)
    

In [8]:
x=data['review']
y=data['label']
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=48)

In [9]:
tf = TfidfVectorizer()
from sklearn.pipeline import Pipeline

In [12]:
from sklearn.linear_model import LogisticRegression
classifier=LogisticRegression()
model=Pipeline([('vectorizer',tf),('classifier',classifier)])

model.fit(x_train,y_train)

C:\Users\HP\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', LogisticRegression())])

In [13]:
vectorizer = TfidfVectorizer(max_df=2, min_df=1, stop_words='english', use_idf=True)
X_train = vectorizer.fit_transform(x_train)
print(vectorizer.get_feature_names(),end=" ")

['aaaa', 'aaaaa', 'aaaaaa', 'aaaaaaa', 'aaaaaaaa', 'aaaaaaaaa', 'aaaaaaaaaaahhhhhhhhhhhhh', 'aaaaaaaarghbr', 'aaaaaaah', 'aaaaaaarrrrrrggggggghhhhhhhbr', 'aaaaaaawwwwwwhhhhhh', 'aaaaaadvertis', 'aaaaaagh', 'aaaaaahhhhh', 'aaaaaghh', 'aaaaah', 'aaaaanyway', 'aaaaargh', 'aaaaarrrrggggg', 'aaaaarrrrgh', 'aaaaarrrrrgggghhh', 'aaaaarrrrrrrrrrggghhhhhhhhhh', 'aaaah', 'aaaahh', 'aaaak', 'aaaar', 'aaaarrgh', 'aaab', 'aaac', 'aaaccategori', 'aaaghh', 'aaahdidnt', 'aaahhh', 'aaahit', 'aaand', 'aaarrrrggggghhhhhh', 'aaarti', 'aaaugh', 'aaaw', 'aab', 'aachen', 'aacquaint', 'aada', 'aadmit', 'aagain', 'aagh', 'aaghh', 'aahhh', 'aaip', 'aaj', 'aaker', 'aakhir', 'aakrosh', 'aaliyah', 'aamesbr', 'aamess', 'aamirpr', 'aamirwel', 'aandhiyaan', 'aaniima', 'aankh', 'aapka', 'aapkey', 'aapki', 'aapko', 'aarc', 'aardvaark', 'aardvark', 'aaronjohnathon', 'aarp', 'aarrrgh', 'aarrrrgh', 'aarti', 'aartidevi', 'aasif', 'aata', 'aaugh', 'aauugghh', 'aav', 'aavjo', 'aaww', 'aawww', 'aay', 'aaya', 'aaye', 'aazmi', 

In [14]:
y_pred=model.predict(x_test)

In [15]:
accuracy_score(y_pred,y_test)

0.7064533333333334

In [16]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.16      0.56      0.24       878
           1       0.32      0.78      0.45      1343
           2       0.95      0.71      0.81     16529

    accuracy                           0.71     18750
   macro avg       0.48      0.68      0.50     18750
weighted avg       0.87      0.71      0.76     18750

